## Lab 4 - Cross-encoder re-ranking

In [ ]:
# from helper_utils import load_chroma, word_wrap, project_embeddings
# from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
# import numpy as np

In [ ]:
# embedding_function = SentenceTransformerEmbeddingFunction()

# chroma_collection = load_chroma(filename='microsoft_annual_report_2022.pdf', collection_name='microsoft_annual_report_2022', embedding_function=embedding_function)
# chroma_collection.count()

In [1]:
import os
import sys
sys.path.append('../..')
import chromadb
import numpy as np
import umap.umap_ as umap

from helper_utils_02 import load_pdf_chroma, word_wrap, project_embeddings,flatten
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from py3810.myUtils import pickle_dump, pickle_load

In [2]:
# Specify the directory to store the database files
path_chromadb_dump = '../Chromadb/'
persist_dir = '../Chromadb/collections/'
is_persistent = True

filename_pdf = 'microsoft_annual_report_2022.pdf'
collection_name = 'microsoft_annual_report_2022'
embedding_function = SentenceTransformerEmbeddingFunction()  # creates 384 dimension vectors

In [3]:
if is_persistent:
  try:
    client = chromadb.PersistentClient(path=persist_dir)
    chroma_collection = client.get_collection(name=collection_name, embedding_function=embedding_function)
    embeddings = pickle_load(filename_pickle='_embeddings', path_pickle_dump=path_chromadb_dump)
    umap_transform = pickle_load(filename_pickle='_umap_transform', path_pickle_dump=path_chromadb_dump)
    projected_dataset_embeddings = pickle_load(filename_pickle='_projected_dataset_embeddings', path_pickle_dump=path_chromadb_dump)
    print(f'loaded embeddings, umpa_transform, projected_dataset_embeddings')
  except FileNotFoundError:
    chroma_collection = \
      load_pdf_chroma(
        filename_pdf=filename_pdf,
        collection_name=collection_name,
        embedding_function=embedding_function,
        persist_directory=persist_dir,
        is_persistent=is_persistent,
      )
    print(f'Created chroma_collection {collection_name}')
    print(f'chroma_collection.count(): {chroma_collection.count()}')
    embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
    umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
    projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)
    print(f'created embeddings, umpa_transform, projected_dataset_embeddings')
    pickle_dump(file_to_pickle=embeddings, filename_pickle='_embeddings', path_pickle_dump=path_chromadb_dump)
    pickle_dump(file_to_pickle=umap_transform, filename_pickle='_umap_transform', path_pickle_dump=path_chromadb_dump)
    pickle_dump(file_to_pickle=projected_dataset_embeddings, filename_pickle='_projected_dataset_embeddings', path_pickle_dump=path_chromadb_dump)    
    print(f'saved embeddings, umpa_transform, projected_dataset_embeddings')
else:
  if os.path.exists(persist_dir):
    client = chromadb.PersistentClient(path=persist_dir)
    chroma_collection = client.get_collection(name=collection_name, embedding_function=embedding_function)
    print(f'Loaded chroma_collection {collection_name}')
    embeddings = pickle_load(filename_pickle='_embeddings', path_pickle_dump=path_chromadb_dump)
    umap_transform = pickle_load(filename_pickle='_umap_transform', path_pickle_dump=path_chromadb_dump)
    projected_dataset_embeddings = pickle_load(filename_pickle='_projected_dataset_embeddings', path_pickle_dump=path_chromadb_dump)
    print(f'loaded embeddings, umpa_transform, projected_dataset_embeddings')
    print(f'chroma_collection.count(): {chroma_collection.count()}')
  else:
    chroma_collection = \
      load_pdf_chroma(
        filename_pdf=filename_pdf,
        collection_name=collection_name,
        embedding_function=embedding_function,
        persist_directory=persist_dir,
        is_persistent=True,  # Set True to save collection, can always delete directory
      )
    print(f'Created chroma_collection {collection_name}')
    print(f'chroma_collection.count(): {chroma_collection.count()}')
    embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
    umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
    projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)
    print(f'created embeddings, umpa_transform, projected_dataset_embeddings')
    pickle_dump(file_to_pickle=embeddings, filename_pickle='_embeddings', path_pickle_dump=path_chromadb_dump)
    pickle_dump(file_to_pickle=umap_transform, filename_pickle='_umap_transform', path_pickle_dump=path_chromadb_dump)
    pickle_dump(file_to_pickle=projected_dataset_embeddings, filename_pickle='_projected_dataset_embeddings', path_pickle_dump=path_chromadb_dump)    
    print(f'saved embeddings, umpa_transform, projected_dataset_embeddings')        

loaded embeddings, umpa_transform, projected_dataset_embeddings


# Re-ranking the long tail

In [4]:
query = "What has been the investment in research and development?"
results = chroma_collection.query(query_texts=query, n_results=10, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

assessment of key technology trends, we maintain our long - term
commitment to research and development across a wide spectrum of
technologies, tools, and platforms spanning digital work and life
experiences, cloud computing, ai, devices, and operating systems. while
our main product research and development facilities are located in
redmond, washington, we also operate research and development
facilities in other parts of the u. s. and around the world. this
global approach helps us remain competitive in local markets and
enables us to continue to attract top talent from across the world. we
plan to continue to make significant investments in a broad range of
product research and development activities, and as appropriate we will
coordinate our research and development across operating segments and
leverage the results across the company. in addition to our main
research and development operations, we also operate microsoft
research. microsoft research is

and business methods, we bel

In [5]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [7]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print(f"len(scores): {len(scores)}")
print("scores:")
for score in scores:
    print(score)

len(scores): 10
scores:
1.6141684
-0.0895655
-2.7713428
-10.692507
-10.671734
-9.126478
-0.40982634
-11.017633
-1.2966429
-8.666394


In [8]:
print("New Ordering:")
# if the index for the largest number is 5, than the 1st number printed is 5
for o in np.argsort(scores)[::-1]:
    print(o)

New Ordering:
0
1
6
8
2
9
5
4
3
7


# Re-ranking with Query Expansion

In [10]:
original_query = "What were the most important factors that contributed to increases in revenue?"
generated_queries = [
    "What were the major drivers of revenue growth?",
    "Were there any new product launches that contributed to the increase in revenue?",
    "Did any changes in pricing or promotions impact the revenue growth?",
    "What were the key market trends that facilitated the increase in revenue?",
    "Did any acquisitions or partnerships contribute to the revenue growth?"
]

In [29]:
n_results = 10 # number retrieved query results  
queries = [original_query] + generated_queries

results = chroma_collection.query(query_texts=queries, n_results=n_results, include=['documents', 'embeddings'])
retrieved_documents = results['documents']
retrieved_embeddings = results['embeddings']

In [40]:
len(retrieved_documents[0][2])

853

In [42]:
len(retrieved_embeddings[0][2])

384

In [31]:
retrieved_documents = results['documents']
retrieved_embeddings = results['embeddings']

print(f'There are {len(queries)} queries')
print(f'The queries retrieved {len(retrieved_documents)} retrieved_documents and {len(retrieved_documents)} retrieved_embeddings')
print(f'Each retrieved document has {len(retrieved_documents[0])} text chunks retrieved from chroma collection')
print(f'Each retrieved embedding has {len(retrieved_embeddings[0])} embedding vectors from chroma collection')

There are 6 queries
The queries retrieved 6 retrieved_documents and 6 retrieved_embeddings
Each retrieved document has 10 text chunks retrieved from chroma collection
Each retrieved embedding has 10 embedding vectors from chroma collection


In [ ]:
# Deduplicate the retrieved documents
document_count = 0
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        document_count += 1
        unique_documents.add(document)

unique_documents = list(unique_documents)
print(f'retrieved {document_count} documents, {len(unique_documents)} are unique')

In [ ]:
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])

In [ ]:
scores = cross_encoder.predict(pairs)

In [ ]:
print(f"len(scores): {len(scores)}")
print("Scores:")
for score in scores:
    print(score)

In [ ]:
print("New Ordering:")
# if the index for the largest number is 5, than the 1st number printed is 5
for o in np.argsort(scores)[::-1]:
    print(o)

In [ ]:
#TODO map the first 5

In [ ]:
n = 5
top_n_scores = np.argsort(scores)[::-1][:n]
top_n_scores

In [ ]:
pairs[17]

In [ ]:
print(F'Top {n} pair score for query: {pairs[0][0]}:')
for i, item in enumerate(top_n_scores):
  print(f'{i} score: {cross_encoder.predict(pairs[item])}')
  print(f'retrived doc: {pairs[item][1]}')

In [ ]:

for i, q in enumerate(queries):
  print(f'{i}: {len(retrieved_documents[i])} {retrieved_documents[i]}')
  print(f'{i}: {len(retrieved_embeddings[i])} {retrieved_embeddings[i]}')

In [ ]:
for doc in retrieved_documents:
    print(word_wrap(doc))
    print('')

In [ ]:
retrieved_embeddings = results['embeddings'][0]
original_query_embedding = embedding_function([original_query])
augmented_query_embedding = embedding_function([joint_query])

projected_original_query_embedding = project_embeddings(original_query_embedding, umap_transform)
projected_augmented_query_embedding = project_embeddings(augmented_query_embedding, umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)